In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import pandas as pd

url = "https://www.daraz.pk/"
# Use Selenium to open the webpage
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))  # Make sure to provide the path to chromedriver if not in PATH
driver.get(url)
driver.find_element(By.ID, "q").send_keys("mobile")
driver.find_element(By.CLASS_NAME, "search-box__button--1oH7").click()
# Wait for the page to load
time.sleep(5)

# Scroll down to load more products
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)

ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

# Loop through each ad card, click on it, and then return to the previous page
price_mobile = []
delivery_status = []
ratings_list = []
seller_ratings_list = []
shipping_performance = []
flagship_mall = []
scores = []
product_name = []
reviews = []
specifications = []
brands = []
index_num = 0
for index_num in range(5):
    if index_num > 0:
        buttons = driver.find_element(By.CLASS_NAME, 'box--eTFFd')
        button_xpath = f'/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div/div/ul/li[{index_num + 2}]'
        button = buttons.find_element(By.XPATH, button_xpath)
        button.click()
        time.sleep(2)

    for index in range(0, 39):
        # Find the ad cards again to avoid StaleElementReferenceException
        ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
        ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

        # Click on the current ad card
        current_card = ad_cards[index]
        driver.execute_script("arguments[0].scrollIntoView();", current_card)
        current_card.click()
        time.sleep(3)

        try:
            Information_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]')
        except NoSuchElementException:
            price_mobile.append('Rs 0')

        prices = Information_grid.find_elements(By.CLASS_NAME, 'pdp-product-price')
        price_ = Information_grid.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span')
        price_mobile.append(price_.text)
        time.sleep(2)
        brand = Information_grid.find_element(By.XPATH,'//*[@id="module_product_brand_1"]/div/a[1]')
        brands.append(brand.text)
        time.sleep(2)
        dilivery_bar = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]')
        dilivery_opt = driver.find_elements(By.CLASS_NAME, 'delivery-option-item__title')
        Seller_rating = dilivery_bar.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
        Shipping_perf = dilivery_bar.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]')
        flagship_mall_xpath = '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img'
        name_of_product = Information_grid.find_element(By.XPATH, '//*[@id="module_product_title_1"]/div/div')
        product_name.append(name_of_product.text)
        flagship_mall_element = dilivery_bar.find_elements(By.XPATH, flagship_mall_xpath)
        if flagship_mall_element:
            flagship_mall.append(1)
        else:
            flagship_mall.append(0)
        seller_ratings_list.append(Seller_rating.text)
        shipping_performance.append(Shipping_perf.text)
        driver.execute_script("window.scrollTo(0, 500);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(500, 800);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(800, 1000);")
        time.sleep(2)
        rating_bar = driver.find_element(By.XPATH, '//*[@id="module_product_review"]')
        rating = rating_bar.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[2]/div[1]/div[1]/div[1]/span[1]')
        time.sleep(2)
        ratings_list.append(rating.text)
        time.sleep(2)
        specification_content = driver.find_element(By.XPATH,'//*[@id="module_product_detail"]/div')
        specification_ = specification_content.find_element(By.XPATH,'//*[@id="module_product_detail"]/div/div/div[1]/ul/li')
        specifications.append(specification_.text)
        time.sleep(2)
        review_content = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]')
        reviews_ = review_content.find_elements(By.XPATH,'//*[@id="module_product_review"]/div/div/div[3]/div[1]/div[2]')
        for review in reviews_:
            reviews.append(review.text)
        counter = 0
        for i in dilivery_opt:
            if 'Free Delivery' in (i.text):
                counter += 1
                delivery_status.append(1)
        if counter == 0:
            delivery_status.append(0)

        # Calculate the base score (price)
        base_score = int(price_mobile[index].replace('Rs. ', '').replace(',', ''))

        # Check if the product has "Daraz Mall" status
        daraz_mall_score = 5000 if flagship_mall[index] == 1 else 0

        # Check if the product has free shipping
        free_shipping_score = 1000 if delivery_status[index] == 1 else 0
        product_rating = float(ratings_list[index])

        time.sleep(2)  # Add a short delay to allow the page to load

        # Go back to the previous page
        driver.back()
        time.sleep(2)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))
driver.quit()
# Print the details of the best product
# Combine data into a dictionary
data = {
    'ID': list(range(1, len(ratings_list) + 1)),
    'Rating': ratings_list,
    'Name': product_name,
    'Price': price_mobile,
    'Specifications': specifications,
    'Brand': brands,
    'Reviews': reviews
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('product.csv', index=False)

# Print the DataFrame
df



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div/div[3]/div[2]/div/div[1]"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x011572A3+45731]
	(No symbol) [0x010E2D51]
	(No symbol) [0x00FD880D]
	(No symbol) [0x0100B940]
	(No symbol) [0x0100BE0B]
	(No symbol) [0x0103D1F2]
	(No symbol) [0x01028024]
	(No symbol) [0x0103B7A2]
	(No symbol) [0x01027DD6]
	(No symbol) [0x010031F6]
	(No symbol) [0x0100439D]
	GetHandleVerifier [0x01460716+3229462]
	GetHandleVerifier [0x014A84C8+3523784]
	GetHandleVerifier [0x014A214C+3498316]
	GetHandleVerifier [0x011E1680+611968]
	(No symbol) [0x010ECCCC]
	(No symbol) [0x010E8DF8]
	(No symbol) [0x010E8F1D]
	(No symbol) [0x010DB2C7]
	BaseThreadInitThunk [0x7651FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D97C6E+28